In [45]:
import numpy as np
from scipy.linalg import eigh
import pandas as pd
from pyspark.sql import SparkSession
from scipy.spatial import KDTree
from pyspark.sql.functions import udf
from pyspark.sql.types import *

In [20]:
def read_csv_point_cloud(csv_file):
    data = pd.read_csv(csv_file, header=None)
    return data

In [21]:
csv_file = "../../Impl/Data/Vaihingen/area2_cov_multi.csv"
points = read_csv_point_cloud(csv_file)

In [22]:
points

,0
0,0.00138718 1.10728 0.0171147 0 0.112454 0.8875...
1,0.00138718 1.10974 0.0171147 0 0.0746411 0.925...
2,0.00154131 1.11221 0.0171147 0 0.0746411 0.925...
3,0.00154131 1.11221 0.0171147 0 0.0746411 0.925...
4,0.00354501 1.11714 0.0171147 0 0.493422 0.5065...
...,...
266670,1.06612 1.76079 0.0214057 0.30497 0.385871 0.3...
266671,1.06628 1.76079 0.0171147 0.270976 0.440006 0....
266672,1.06643 1.76326 0.0171147 0.29536 0.318112 0.3...
266673,1.06643 1.76572 0.0171147 0.233511 0.364512 0....


In [23]:
points[['x', 'y', 'z', 'cl', 'cs', 'cp', 'label']] = points[0].str.split(' ', expand=True)
points

,0,x,y,z,cl,cs,cp,label
0,0.00138718 1.10728 0.0171147 0 0.112454 0.8875...,0.00138718,1.10728,0.0171147,0,0.112454,0.887546,2
1,0.00138718 1.10974 0.0171147 0 0.0746411 0.925...,0.00138718,1.10974,0.0171147,0,0.0746411,0.925359,2
2,0.00154131 1.11221 0.0171147 0 0.0746411 0.925...,0.00154131,1.11221,0.0171147,0,0.0746411,0.925359,2
3,0.00154131 1.11221 0.0171147 0 0.0746411 0.925...,0.00154131,1.11221,0.0171147,0,0.0746411,0.925359,2
4,0.00354501 1.11714 0.0171147 0 0.493422 0.5065...,0.00354501,1.11714,0.0171147,0,0.493422,0.506578,2
...,...,...,...,...,...,...,...,...
266670,1.06612 1.76079 0.0214057 0.30497 0.385871 0.3...,1.06612,1.76079,0.0214057,0.30497,0.385871,0.309159,5
266671,1.06628 1.76079 0.0171147 0.270976 0.440006 0....,1.06628,1.76079,0.0171147,0.270976,0.440006,0.289018,8
266672,1.06643 1.76326 0.0171147 0.29536 0.318112 0.3...,1.06643,1.76326,0.0171147,0.29536,0.318112,0.386527,8
266673,1.06643 1.76572 0.0171147 0.233511 0.364512 0....,1.06643,1.76572,0.0171147,0.233511,0.364512,0.401977,8


In [24]:
points.drop(columns=[0], inplace=True)

In [25]:
points

,x,y,z,cl,cs,cp,label
0,0.00138718,1.10728,0.0171147,0,0.112454,0.887546,2
1,0.00138718,1.10974,0.0171147,0,0.0746411,0.925359,2
2,0.00154131,1.11221,0.0171147,0,0.0746411,0.925359,2
3,0.00154131,1.11221,0.0171147,0,0.0746411,0.925359,2
4,0.00354501,1.11714,0.0171147,0,0.493422,0.506578,2
...,...,...,...,...,...,...,...
266670,1.06612,1.76079,0.0214057,0.30497,0.385871,0.309159,5
266671,1.06628,1.76079,0.0171147,0.270976,0.440006,0.289018,8
266672,1.06643,1.76326,0.0171147,0.29536,0.318112,0.386527,8
266673,1.06643,1.76572,0.0171147,0.233511,0.364512,0.401977,8


In [26]:
points_arr = points[['x', 'y', 'z']].to_numpy(dtype=np.float64)
points_arr

array([[1.38718e-03, 1.10728e+00, 1.71147e-02],
       [1.38718e-03, 1.10974e+00, 1.71147e-02],
       [1.54131e-03, 1.11221e+00, 1.71147e-02],
       ...,
       [1.06643e+00, 1.76326e+00, 1.71147e-02],
       [1.06643e+00, 1.76572e+00, 1.71147e-02],
       [1.06967e+00, 1.76326e+00, 1.71147e-02]])

In [27]:
def compute_geometric_features(points_arr):
    cov_matrix =np.cov(points_arr, rowvar=False)
    eigenvalues, eigenvectors = eigh(cov_matrix)
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    omnivariance = np.prod(eigenvalues) ** (1/3)
    eigenentropy = -np.sum(eigenvalues * np.log(eigenvalues))
    anisotropy = (eigenvalues[0] - eigenvalues[2]) / eigenvalues[0]
    linearity = (eigenvalues[0] - eigenvalues[1]) / eigenvalues[0]
    planarity = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0]
    scattering = eigenvalues[2] / eigenvalues[0]
    sum_of_eigenvalues = np.sum(eigenvalues)
    change_of_curvature = eigenvalues[2] / np.sum(eigenvalues)
    return omnivariance, eigenentropy, anisotropy, linearity, planarity, scattering, sum_of_eigenvalues, change_of_curvature


In [28]:
print(compute_geometric_features(points_arr))

(0.014169021351177946, 0.3410513744379214, 0.9885157805234319, 0.3244220972197599, 0.664093683303672, 0.011484219476568159, 0.12074763212830705, 0.006807229754649193)


In [30]:
def find_k_closest_points(points, query_point, k):
    kdtree = KDTree(points)
    _, indices = kdtree.query(query_point, k) # Returns distances and indices of the k-closest points to the query points.

    return indices

In [31]:
def find_points_within_radius(points, query_point, r):
    kdtree = KDTree(points)
    indices = kdtree.query_ball_point(query_point, r) # Returns indices of all points within distance r from the query point.
    return indices

In [32]:
len(find_points_within_radius(points_arr, points_arr[3222], 0.2))

27747

In [35]:
import scipy

In [38]:
import os
import sys

os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

In [54]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Convert numpy array to PySpark DataFrame
points_df = spark.createDataFrame(points)
# Define UDF to compute geometric features
@udf(returnType=ArrayType(DoubleType()))
def compute_geometric_features_udf(point, neighbors):
    # Compute geometric features using point and neighbors
    # Replace this with your actual implementation
    return [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
# Find neighbors within radius 0.25 for each point
neighbors_df = points_df.rdd.map(lambda row: (row, find_points_within_radius(points_arr, row, 0.25))).toDF(["point", "neighbors"])
# Apply UDF to compute geometric features using neighbors
geometric_features_df = neighbors_df.withColumn("geometric_features",
    compute_geometric_features_udf(neighbors_df["point"], neighbors_df["neighbors"]))
# Collect the results as a new DataFrame
result_df = geometric_features_df.select("point", "geometric_features")

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 225) (Vidhish-HP executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\Vidhish17\Desktop\RE\gvcl\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1227, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Vidhish17\Desktop\RE\gvcl\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 90, in read_command
    command = serializer._read_with_length(file)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Vidhish17\Desktop\RE\gvcl\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serial